# Trump Speech Generator (tweepy, gpt-2)

* The purpose of this notebook is to explore using: (1) Tweepy for downloading data from the Twitter API; and (2) GPT-2 for generating new text.

**Step 1 of 5:** We will begin by installing the Python library "[tweepy](https://www.tweepy.org/)" to interact with the Twitter API.

In [1]:
!pip install tweepy

     |████████████████████████████████| 153kB 9.8MB/s 


**Step 2 of 5:** We will also use the "user secrets" functionality on Kaggle in order to manage our private API keys.  You can manage your "secrets" in the "add-ons" menu in the upper-most file menu within the [notebook editor](http://www.kaggle.com/kernels).  To interact with the Twitter API you will need to have an ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, and CONSUMER_SECRET.

In [2]:
from kaggle_secrets import UserSecretsClient
ACCESS_TOKEN = UserSecretsClient().get_secret('ACCESS_TOKEN')
ACCESS_SECRET = UserSecretsClient().get_secret('ACCESS_SECRET')
CONSUMER_KEY = UserSecretsClient().get_secret('CONSUMER_KEY')
CONSUMER_SECRET = UserSecretsClient().get_secret('CONSUMER_SECRET')

**Step 3 of 5:** We also need to download the [GPT-2 repository]( https://github.com/graykode/gpt-2-Pytorch).

In [3]:
import os
!git clone https://github.com/graykode/gpt-2-Pytorch.git
os.chdir('./gpt-2-Pytorch')
!curl --output gpt2-pytorch_model.bin https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
!pip install -r requirements.txt

Cloning into 'gpt-2-Pytorch'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 130 (delta 0), reused 0 (delta 0), pack-reused 129
Receiving objects: 100% (130/130), 2.39 MiB | 0 bytes/s, done.
Resolving deltas: 100% (48/48), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  522M  100  522M    0     0  69.9M      0  0:00:07  0:00:07 --:--:-- 69.2M
     |████████████████████████████████| 604kB 3.5MB/s 
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533339 sha256=89b89724cf8bfb6433b1ab18209c9db7434935fb04fa185305b2626cd22b27ec
  Stored in directory: /tmp/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built regex
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
E

**Step 4 of 5:** Next I will create a dataframe of tweets from @realDonaldTrump
* Twitter's [Developer Agreement and Developer Policy](https://developer.twitter.com/en/developer-terms/agreement-and-policy.html) are both very strict and include terms such as "If you provide Twitter Content to third parties, including downloadable datasets of Twitter Content or an API that returns Twitter Content, you will only distribute or allow download of Tweet IDs, Direct Message IDs, and/or User IDs.” In an attempt to comply with this policy I expose only recent tweets from @realdonaldtrump (which are already a part of the official Presidential Records).

In [4]:
import tweepy
import pandas as pd

def connect_to_twitter_OAuth():
    """adapted from https://towardsdatascience.com/my-first-twitter-app-1115a327349e"""
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(auth)
    return api

def extract_tweets(tweet_object):
    """adapted from https://towardsdatascience.com/my-first-twitter-app-1115a327349e"""
    tweet_list =[]
    for tweet in tweet_object:
        text = tweet.text
        created_at = tweet.created_at
        tweet_list.append({'created_at':created_at,'text':text,})
    df = pd.DataFrame(tweet_list, columns=[ 'created_at','text',])
    return df

In [5]:
api = connect_to_twitter_OAuth()
trump_tweets = api.user_timeline('realdonaldtrump')
df = extract_tweets(trump_tweets)
pd.set_option('display.max_colwidth', -1)
df.tail(10)

,created_at,text
9,2019-10-21 01:57:10,Congratulations Barbara! https://t.co/0cYrjL1YOj
10,2019-10-21 01:56:05,"RT @SecretaryRoss: Since the election of President Trump, the U.S. has added 6.4 million jobs, including 500,000 in the #manufacturing sect…"
11,2019-10-21 01:55:36,RT @SecretaryRoss: This action by the Commerce Department sends another clear message to the Cuban regime – that they must immediately ceas…
12,2019-10-21 01:54:14,RT @realDonaldTrump: It is ONLY about this! https://t.co/ZB5xPDKs4b
13,2019-10-21 01:52:11,RT @IngrahamAngle: Republicans in the Congress should be asked: why have you allowed the State Department to become an arm of the Democrat…
14,2019-10-21 01:48:35,"“To the Democrats, Impeachment is partisan politics dressed up as principle.” @SteveHiltonx @FoxNews"
15,2019-10-20 22:41:10,"So interesting that, when I announced Trump National Doral in Miami would be used for the hosting of the G-7, and t… https://t.co/tDDMw6CPOd"
16,2019-10-20 21:45:17,"....fiction to Congress and the American People? I demand his deposition. He is a fraud, just like the Russia Hoax… https://t.co/Hum7mHfNiV"
17,2019-10-20 21:45:17,....because their so-called story didn’t come even close to matching up with the exact transcript of the phone call… https://t.co/SUGMgGZeJ3
18,2019-10-20 21:45:17,"This Scam going on right now by the Democrats against the Republican Party, and me, was all about a perfect phone c… https://t.co/nXPfrl25ag"


**Step 5 of 5:** And finally I will take the list of tweets that we retrieved and I will feed a few of them into a [GPT-2](https://github.com/graykode/gpt-2-Pytorch) model to generate a full-blown speech.

In [6]:
!python main.py --text "To the Democrats, Impeachment is partisan politics dressed up as principle.  This Scam going on right now by the Democrats against the Republican Party, and me, was all about a perfect phone call."

Namespace(batch_size=-1, length=-1, nsamples=1, quiet=False, temperature=0.7, text='To the Democrats, Impeachment is partisan politics dressed up as principle.  This Scam going on right now by the Democrats against the Republican Party, and me, was all about a perfect phone call.', top_k=40, unconditional=False)
To the Democrats, Impeachment is partisan politics dressed up as principle.  This Scam going on right now by the Democrats against the Republican Party, and me, was all about a perfect phone call.
100%|█████████████████████████████████████████| 512/512 [00:29<00:00, 17.42it/s]
======================================== SAMPLE 1 ========================================
  I will tell you, it was a perfect call.  It went on for about 10 minutes, and the Democrats have called me on a number of different issues, including a number of issues that I really don't like.  They call me on that and that, and that, all around being very, very respectful. Then I'm told the Democrats have calle

In [7]:
!rm -r /kaggle/working/*